In [ ]:
# watch this video

# https://www.youtube.com/watch?v=L9BLLWRtnOU

In [46]:
# import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [75]:
df = pd.read_csv('/Users/emina/Downloads/heart.csv', sep=',')

# Describe dataset

In [48]:
df.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [50]:
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

# Possible preprocessing steps

In [51]:
# df['classes'] = df['classes'].map({4:1, 2:0})

In [52]:
# df.drop('id',axis=1,inplace=True)
# df.drop('Unnamed: 32',axis=1,inplace=True)

In [53]:
# data[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = data[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)

In [54]:
#def median_target(var):   
#    temp = data[data[var].notnull()]
#    temp = temp[[var, 'Outcome']].groupby(['Outcome'])[[var]].median().reset_index()
#    return temp

In [55]:
# df = df[df.isnull()==True]
# df['Age'] = df['Age'].interpolate()
# df = df.dropna()

# Prepare libraries and split into predictors and target

In [76]:
# from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [77]:
X = df.drop(['target'], axis=1)
y = df['target']

# Feature Selection

In [78]:
from sklearn.feature_selection import SelectKBest, chi2, f_regression

fs = SelectKBest(score_func=chi2, k=6)
X_kbest = fs.fit_transform(X, y)
columns = list(X.columns[fs.get_support(indices=True)])

X = pd.DataFrame(X_kbest)
X.columns = columns

In [79]:
X

,cp,chol,thalach,exang,oldpeak,ca
0,3.0,233.0,150.0,0.0,2.3,0.0
1,2.0,250.0,187.0,0.0,3.5,0.0
2,1.0,204.0,172.0,0.0,1.4,0.0
3,1.0,236.0,178.0,0.0,0.8,0.0
4,0.0,354.0,163.0,1.0,0.6,0.0
...,...,...,...,...,...,...
298,0.0,241.0,123.0,1.0,0.2,0.0
299,3.0,264.0,132.0,0.0,1.2,0.0
300,0.0,193.0,141.0,0.0,3.4,2.0
301,0.0,131.0,115.0,1.0,1.2,1.0


# Hyperparameter tuning with GridCV

In [81]:
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
test_sizes = [0.2, 0.25, 0.3, 0.35, 0.4]
best_score = 0
best_size = test_sizes[0]
best_params = []

In [83]:
for ts in test_sizes:
    
    print('\nTest size: ', ts)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ts, random_state=0)
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_test = ss.fit_transform(X_test)
    
    # define grid search
    grid = dict(solver=solvers, penalty=penalty, C=c_values)

    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy', error_score=0)
    grid_result = grid_search.fit(X_train, y_train)
    
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
        if mean > best_score:
            best_score = mean
            best_size = ts
            best_params = grid_result.best_params_

print('Best score and test size: (' + str(best_score) + ', ' + str(best_size) + ')' )


Test size:  0.2
Best: 0.815556 using {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}
0.811389 (0.090504) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.811389 (0.090504) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.811389 (0.090504) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.811389 (0.090504) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.811389 (0.090504) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.811389 (0.090504) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.810000 (0.090476) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.810000 (0.090476) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.810000 (0.090476) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.808667 (0.094602) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.808667 (0.094602) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.814111 (0.090707) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.815

# Testing model performance

In [84]:
results_LR = pd.DataFrame([], columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

In [85]:
# make classifier and train model
classifier = LogisticRegression(random_state=0, penalty=best_params.get('penalty'), solver=best_params.get('solver'), C=best_params.get('C'))
model = classifier.fit(X_train, y_train)
y_pred = model.predict(X_test)

# calculate accuracy on train set
y_pred_train = classifier.predict(X_train)
acc_train = accuracy_score(y_train, y_pred_train)

# calculate scores on test set
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# record the results
model_results = pd.DataFrame([['Logistic Regression', acc_train, acc, prec, rec, f1]],
               columns = ['Model', 'Train set acc.', 'Test set acc.', 'Precision', 'Recall', 'F1 Score'])
results_LR = results_LR.append(model_results, ignore_index = True)

model_results

,Model,Train set acc.,Test set acc.,Precision,Recall,F1 Score
0,Logistic Regression,0.867403,0.770492,0.739726,0.857143,0.794118


# Cross-validation

In [90]:
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std

scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.808 (0.071)
